doplnit graf zobrazujici data porizeni snimku

In [2]:
# import libraries
import sentinelsat
import pyroSAR
from osgeo import ogr
import geopandas
import matplotlib.pyplot as plt

from os import listdir
from os.path import join

In [5]:
ROOT_DIR = 'f:/datasets/NATUR_CUNI'

In [ ]:
# read credentials from file
def read_credentials(filename):
    with open(filename, 'r') as txtfile:
        username, password = txtfile.read().split()
        return username, password

username, password = read_credentials('credentials_api_hub.txt')
print(username)
print(password)

In [ ]:
# parameters
roi = 'POLYGON((15.425 50.74, 15.515 50.74, 15.515 50.635, 15.425 50.635, 15.425 50.74))'
seasons = (('20210401', '20210930'), ('20220401', '20220930'))
orbits = (22, 95, 73, 146)

In [ ]:
# download files
for season in seasons:
    for orbit in orbits:
        query_result = api.query(
            date=season,
            platformname='Sentinel-1',
            raw=None,
            area=roi,
            area_relation='Contains',
            order_by=None,
            limit=200,
            offset=0,
            relativeorbitnumber=orbit,
            producttype='SLC',
            sensoroperationalmode='IW')
        print(f'Query for RO {orbit} in {season[0][:4]} returned {len(query_result)} responses.')
        download_dir = join(ROOT_DIR, f's1_download/{season[0][:4]}/{orbit}')
        api.download_all(query_result, directory_path=download_dir, n_concurrent_dl=4)

In [ ]:
# check files
for season in seasons:
    for orbit in orbits:
        download_dir = join(ROOT_DIR, f's1_download/{season[0][:4]}/{orbit}')
        download_list = [join(download_dir, filename) for filename in listdir(download_dir)]
        broken_files = api.check_files(paths=download_list, delete=False).keys()
        print(f'Files from {season[0][:4]}, RO {orbit} have {len(broken_files)} broken files.')

In [7]:
osv = pyroSAR.S1.OSV(osvdir=join(ROOT_DIR, 's1_orbs'))

In [11]:
products = osv.catch(['S1A', 'S1B'], osvtype='POE', start='20210402T000000', stop='20210406T000000', url_option=1)
print(products)

In [12]:
osv.retrieve(products, pbar=True)

100% (4 of 4) |##########################| Elapsed Time: 0:00:07 Time:  0:00:07


In [ ]:
for season in seasons:
    for orbit in orbits:
        download_dir = join(ROOT_DIR, f's1_download/{season[0][:4]}/{orbit}')
        download_list = [join(download_dir, filename) for filename in listdir(download_dir)]

### Possibly deletable?

In [ ]:
# initialise api
api = sentinelsat.SentinelAPI(
    username,
    password,
    api_url='https://apihub.copernicus.eu/apihub/',
    show_progressbars=True,
    timeout=60)

In [ ]:
# query the api
query_result = api.query(
    date=('20220401', '20220930'),
    platformname='Sentinel-1',
    raw=None,
    area=roi,
    area_relation='Contains',
    order_by=None,
    limit=200,
    offset=0,
    relativeorbitnumber=(95),
    producttype='SLC',
    sensoroperationalmode='IW')
print(f'This query returned {len(query_result)} responses.')

In [ ]:
# draw the imagery boundaries
df = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
background = df[df['name'] == 'Czechia'].plot(color='white', edgecolor='black')
geopandas.GeoSeries.from_wkt([roi]).plot(ax=background)

gdf = api.to_geodataframe(query_result)
gdf.boundary.plot(ax=background, marker='o', color='red', markersize=5);

## Download all images

In [ ]:
seasons = (('20210401', '20210930'), ('20220401', '20220930'))
orbits = (22, 95, 73, 146)

In [ ]:
for season in seasons:
    for orbit in orbits:
        query_result = api.query(
            date=season,
            platformname='Sentinel-1',
            raw=None,
            area=roi,
            area_relation='Contains',
            order_by=None,
            limit=200,
            offset=0,
            relativeorbitnumber=orbit,
            producttype='SLC',
            sensoroperationalmode='IW')
        print(f'Query for RO {str(orbit)} in {str(season[0][:4])} returned {len(query_result)} responses.')
        download_dir = f'E:/datasets/NATUR_CUNI/_dp_download/s1/{str(season[0][:4])}/{str(orbit)}'
        api.download_all(query_result, directory_path=download_dir, n_concurrent_dl=4)

In [ ]:
# print capture dates
capture_dates = [i["beginposition"] for i in query_result.values()]
print(capture_dates)

In [ ]:
plt.plot(capture_dates, len(capture_dates)*[0], '+')

In [ ]:
# download everything from the query to folder
api.download_all(query_result, directory_path='E:/datasets/NATUR_CUNI/_dp_download/sentinel1/73', n_concurrent_dl=4)

In [ ]:
help(api.query)

In [ ]:
help(api.check)

In [ ]:
# pseudocode overall structure
download_all
check_correct_download
add_to_db
remove